In [1]:
import pandas as pd
import numpy as np
import random
import math


class Tree(object):
    def __init__(self):
        self.split_feature = None
        self.split_value = None
        self.leaf_value = None
        self.tree_left = None
        self.tree_right = None

    def calc_predict_value(self, dataset):
        if self.leaf_value is not None:
            return self.leaf_value
        elif dataset[self.split_feature] <= self.split_value:
            return self.tree_left.calc_predict_value(dataset)
        else:
            return self.tree_right.calc_predict_value(dataset)

    def describe_tree(self):

        if not self.tree_left and not self.tree_right:
            leaf_info = "{leaf_value:" + str(self.leaf_value) + "}"
            return leaf_info
        left_info = self.tree_left.describe_tree()
        right_info = self.tree_right.describe_tree()
        tree_structure = "{split_feature:" + str(self.split_feature) + \
                         ",split_value:" + str(self.split_value) + \
                         ",left_tree:" + left_info + \
                         ",right_tree:" + right_info + "}"
        return tree_structure


class RandomForestRegression(object):
    def __init__(self, n_estimators=10, max_depth=-1, min_samples_split=2, min_samples_leaf=1,
                 min_split_gain=0.0, colsample_bytree=None, subsample=0.8, random_state=None):

        self.n_estimators = n_estimators
        self.max_depth = max_depth if max_depth != -1 else float('inf')
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_split_gain = min_split_gain
        self.colsample_bytree = colsample_bytree
        self.subsample = subsample
        self.random_state = random_state
        self.trees = None
        self.feature_importances_ = dict()

    def fit(self, dataset, targets):

        targets = targets.to_frame(name='label')

        if self.random_state:
            random.seed(self.random_state)
        random_state_stages = random.sample(range(self.n_estimators), self.n_estimators)

        
        if self.colsample_bytree == "sqrt":
            self.colsample_bytree = int(len(dataset.columns) ** 0.5)
        elif self.colsample_bytree == "log2":
            self.colsample_bytree = int(math.log(len(dataset.columns)))
        else:
            self.colsample_bytree = len(dataset.columns)


        self.trees = []
        for random_state in random_state_stages:
            self.trees.append(self._parallel_build_trees(dataset, targets, random_state))
        
    def _parallel_build_trees(self, dataset, targets, random_state):
      
        subcol_index = random.sample(dataset.columns.tolist(), self.colsample_bytree)
        dataset_stage = dataset.sample(n=int(self.subsample * len(dataset)), replace=True, 
                                        random_state=random_state).reset_index(drop=True)
        dataset_stage = dataset_stage.loc[:, subcol_index]
        targets_stage = targets.sample(n=int(self.subsample * len(dataset)), replace=True, 
                                        random_state=random_state).reset_index(drop=True)

        tree = self._build_single_tree(dataset_stage, targets_stage, depth=0)
        print(tree.describe_tree())
        return tree

    def _build_single_tree(self, dataset, targets, depth):

        if len(targets['label'].unique()) <= 1 or dataset.__len__() <= self.min_samples_split:
            tree = Tree()
            tree.leaf_value = self.calc_leaf_value(targets['label'])
            return tree

        if depth < self.max_depth:
            best_split_feature, best_split_value, best_split_gain = self.choose_best_feature(dataset, targets)
            left_dataset, right_dataset, left_targets, right_targets = \
                self.split_dataset(dataset, targets, best_split_feature, best_split_value)

            tree = Tree()
         
            if left_dataset.__len__() <= self.min_samples_leaf or \
                    right_dataset.__len__() <= self.min_samples_leaf or \
                    best_split_gain <= self.min_split_gain:
                tree.leaf_value = self.calc_leaf_value(targets['label'])
                return tree
            else:
 
                self.feature_importances_[best_split_feature] = \
                    self.feature_importances_.get(best_split_feature, 0) + 1

                tree.split_feature = best_split_feature
                tree.split_value = best_split_value
                tree.tree_left = self._build_single_tree(left_dataset, left_targets, depth+1)
                tree.tree_right = self._build_single_tree(right_dataset, right_targets, depth+1)
                return tree

        else:
            tree = Tree()
            tree.leaf_value = self.calc_leaf_value(targets['label'])
            return tree

    def choose_best_feature(self, dataset, targets):

        best_split_gain = float("inf")
        best_split_feature = None
        best_split_value = None

        for feature in dataset.columns:
            if dataset[feature].unique().__len__() <= 100:
                unique_values = sorted(dataset[feature].unique().tolist())

            else:
                unique_values = np.unique([np.percentile(dataset[feature], x)
                                           for x in np.linspace(0, 100, 100)])

            # 对可能的分裂阈值求分裂增益，选取增益最大的阈值
            for split_value in unique_values:
                left_targets = targets[dataset[feature] <= split_value]
                right_targets = targets[dataset[feature] > split_value]
                split_gain = self.calc_r2(left_targets['label'], right_targets['label'])

                if split_gain < best_split_gain:
                    best_split_feature = feature
                    best_split_value = split_value
                    best_split_gain = split_gain
        return best_split_feature, best_split_value, best_split_gain

    @staticmethod
    def calc_leaf_value(targets):
    
        return targets.mean()

    @staticmethod
    def calc_r2(left_targets, right_targets):

        r2 = 0
        for targets in [left_targets, right_targets]:
            mean = targets.mean()
            for dt in targets:
                r2 += (dt - mean) ** 2
        return r2

    @staticmethod
    def split_dataset(dataset, targets, split_feature, split_value):

        left_dataset = dataset[dataset[split_feature] <= split_value]
        left_targets = targets[dataset[split_feature] <= split_value]
        right_dataset = dataset[dataset[split_feature] > split_value]
        right_targets = targets[dataset[split_feature] > split_value]
        return left_dataset, right_dataset, left_targets, right_targets

    def predict(self, dataset):
    
        res = []
        for _, row in dataset.iterrows():
            pred_list = []
            
            for tree in self.trees:
                pred_list.append(tree.calc_predict_value(row))
            res.append(sum(pred_list) * 1.0 / len(pred_list))
        return np.array(res)

In [15]:
df = pd.read_csv("NTN-data.csv").fillna(-1)
df = df.rename(columns={'pH': 'label'})
clf = RandomForestRegression(n_estimators=5,
                             max_depth=5,
                             min_samples_split=50,
                             min_samples_leaf=10,
                             min_split_gain=0.0,
                             colsample_bytree="sqrt",
                             subsample=0.8,
                             random_state=66)
train_count = int(0.7 * len(df))

feature_list = ['Ca', 'Na', 'NH4', 'SO4', 'Cl', 'K', 'NO3']
clf.fit(df.loc[:train_count, feature_list], df.loc[:train_count, 'label'])

print(np.power((df.loc[:train_count, 'label'] - clf.predict(df.loc[:train_count, feature_list])), 2).sum()/ len(df.loc[:train_count, 'label']))
print(np.power((df.loc[train_count:, 'label'] - clf.predict(df.loc[train_count:, feature_list])), 2).sum()/ len(df.loc[train_count:, 'label']))

from sklearn import metrics
print(metrics.mean_squared_error(df.loc[:train_count, 'label'], clf.predict(df.loc[:train_count, feature_list])))
print(metrics.mean_squared_error(df.loc[train_count:, 'label'], clf.predict(df.loc[train_count:, feature_list])))

{split_feature:NO3,split_value:1.070989898989899,left_tree:{leaf_value:4.9151707317073186},right_tree:{split_feature:NO3,split_value:1.962,left_tree:{split_feature:NH4,split_value:0.44799999999999995,left_tree:{split_feature:NO3,split_value:1.143,left_tree:{leaf_value:4.670857142857143},right_tree:{split_feature:NO3,split_value:1.453,left_tree:{leaf_value:4.477634615384615},right_tree:{leaf_value:4.315800000000001}}},right_tree:{leaf_value:4.786057142857143}},right_tree:{leaf_value:4.19796153846154}}}
{split_feature:NO3,split_value:1.0737474747474751,left_tree:{split_feature:NO3,split_value:0.6553636363636365,left_tree:{split_feature:Cl,split_value:0.11800000000000001,left_tree:{split_feature:NO3,split_value:0.599,left_tree:{leaf_value:5.077863636363636},right_tree:{leaf_value:5.377866666666667}},right_tree:{split_feature:NO3,split_value:0.43700000000000006,left_tree:{leaf_value:5.1106799999999994},right_tree:{leaf_value:4.860692307692307}}},right_tree:{leaf_value:4.821674242424244}},r

In [3]:
clf.feature_importances_

{'NO3': 12, 'NH4': 1, 'Cl': 1, 'Ca': 1, 'SO4': 13, 'Na': 2}

In [38]:
len(df)

702

In [45]:
def get_nfolds_datasets(dataset, n_folds, split=0.9):
    all_datasets_splited_train = []
    all_datasets_splited_test = []
    size_one_fold = len(dataset) / n_folds
    for i in range(n_folds):
        left = int(i * len(dataset) / n_folds)
        right = int((i + 1) * len(dataset) / n_folds)
        all_datasets_splited_test.append(dataset.iloc[left:right])
        all_datasets_splited_train.append(pd.concat([dataset.iloc[:left], dataset.iloc[right:]], ignore_index=True))
    return all_datasets_splited_train, all_datasets_splited_test

In [46]:
all_datasets_splited_train, all_datasets_splited_test = get_nfolds_datasets(df, 10)

In [47]:
all_datasets_splited_test

[   siteID  month    yr  Criteria1  Criteria2  Criteria3     Ca     Mg      K  \
 0    VA24      1  1999         20         20        100  0.041  0.072  0.027   
 1    VA24      2  1999        100        100        100  0.164  0.031  0.041   
 2    VA24      3  1999        100        100        100  0.165  0.026  0.022   
 3    VA24      4  1999         67        100         71  0.070  0.038  0.022   
 4    VA24      5  1999         80        100         99  0.077  0.024  0.019   
 ..    ...    ...   ...        ...        ...        ...    ...    ...    ...   
 65   VA24      8  2004        100        100        100  0.042  0.008  0.007   
 66   VA24      9  2004        100        100        100  0.040  0.031  0.016   
 67   VA24     10  2004        100        100        100  0.045  0.009  0.010   
 68   VA24     11  2004        100        100        100  0.031  0.010  0.007   
 69   VA24     12  2004        100        100        100  0.035  0.018  0.015   
 
        Na  ...    SO4  Br

In [48]:
print(np.power((all_datasets_splited_test[0].loc[:, 'label'] - clf.predict(all_datasets_splited_test[0].loc[:, feature_list])), 2).sum()/ len(all_datasets_splited_test[0].loc[:, 'label']))

0.0396883188219255


In [51]:
df = pd.read_csv("NTN-data.csv").fillna(-1)
df = df.rename(columns={'pH': 'label'})

def cross_validate(df, n_folds=10):
    all_datasets_splited_train, all_datasets_splited_test = get_nfolds_datasets(df, n_folds)
    
    for i in range(n_folds):
        clf = RandomForestRegression(n_estimators=5,
                                     max_depth=5,
                                     min_samples_split=50,
                                     min_samples_leaf=10,
                                     min_split_gain=0.0,
                                     colsample_bytree="sqrt",
                                     subsample=0.8,
                                     random_state=66)
        train_count = int(0.8 * len(df))

        

        feature_list = ['Ca', 'Na', 'NH4', 'SO4', 'Cl', 'K', 'NO3']
        
        clf.fit(all_datasets_splited_train[i].loc[:train_count, feature_list], all_datasets_splited_train[i].loc[:train_count, 'label'])

        print("feature importance:", clf.feature_importances_)

        print(f"fold: {i}, error train:", np.power((all_datasets_splited_train[i].loc[:train_count, 'label'] - clf.predict(all_datasets_splited_train[i].loc[:train_count, feature_list])), 2).sum()/ len(all_datasets_splited_train[i].loc[:train_count, 'label']))
        print(f"fold: {i}, error val:", np.power((all_datasets_splited_train[i].loc[train_count:, 'label'] - clf.predict(all_datasets_splited_train[i].loc[train_count:, feature_list])), 2).sum()/ len(all_datasets_splited_train[i].loc[train_count:, 'label']))
        print(f"fold: {i}, error test:", np.power((all_datasets_splited_test[i].loc[:, 'label'] - clf.predict(all_datasets_splited_test[i].loc[:, feature_list])), 2).sum()/ len(all_datasets_splited_test[i].loc[:, 'label']))

cross_validate(df, 10)


{split_feature:NO3,split_value:1.072,left_tree:{split_feature:NH4,split_value:0.3240707070707073,left_tree:{split_feature:NO3,split_value:0.6210909090909091,left_tree:{split_feature:NH4,split_value:0.25,left_tree:{split_feature:NO3,split_value:0.27,left_tree:{leaf_value:5.285291666666668},right_tree:{leaf_value:5.037880952380953}},right_tree:{leaf_value:5.523636363636364}},right_tree:{split_feature:NO3,split_value:0.755,left_tree:{split_feature:NH4,split_value:0.21100000000000002,left_tree:{leaf_value:4.805137931034483},right_tree:{leaf_value:5.055913043478259}},right_tree:{split_feature:NH4,split_value:0.251,left_tree:{leaf_value:4.646016666666667},right_tree:{leaf_value:4.836967741935484}}}},right_tree:{leaf_value:5.533232558139534}},right_tree:{split_feature:NO3,split_value:1.841,left_tree:{split_feature:NH4,split_value:0.521,left_tree:{split_feature:NO3,split_value:1.5419999999999998,left_tree:{leaf_value:4.603448275862069},right_tree:{leaf_value:4.398176470588235}},right_tree:{lea